In [1]:
import os

# 1. Create a directory for the data
if not os.path.exists('data'):
    os.makedirs('data')

# 2. Download the zip file (using a reliable mirror for the NASA dataset)
!#wget https://data.nasa.gov/docs/legacy/CMAPSSData.zip -O data/CMAPSSData.zip

# 3. Unzip it
#!unzip -o data/CMAPSSData.zip -d data/

#print("Data downloaded and extracted!")

In [2]:
# The dataset has 26 columns
# 1. Unit Number (Which engine is it?)
# 2. Time Cycles (How long has it been running?)
# 3-5. Operational Settings (Altitude, Speed, etc.)
# 6-26. Sensor Readings (s1 to s21)

index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i) for i in range(1, 22)] 
col_names = index_names + setting_names + sensor_names

print(col_names)

['unit_nr', 'time_cycles', 'setting_1', 'setting_2', 'setting_3', 's_1', 's_2', 's_3', 's_4', 's_5', 's_6', 's_7', 's_8', 's_9', 's_10', 's_11', 's_12', 's_13', 's_14', 's_15', 's_16', 's_17', 's_18', 's_19', 's_20', 's_21']


In [3]:
import pandas as pd

data_df = pd.read_csv('data/train_FD001.txt', sep=r'\s+', header=None, names=col_names)

print("Training data shape:", data_df.shape)
data_df.head()

Training data shape: (20631, 26)


,unit_nr,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,s_5,...,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


In [4]:
col_names

['unit_nr',
 'time_cycles',
 'setting_1',
 'setting_2',
 'setting_3',
 's_1',
 's_2',
 's_3',
 's_4',
 's_5',
 's_6',
 's_7',
 's_8',
 's_9',
 's_10',
 's_11',
 's_12',
 's_13',
 's_14',
 's_15',
 's_16',
 's_17',
 's_18',
 's_19',
 's_20',
 's_21']

In [5]:
max_value_cycle = data_df.groupby('unit_nr')['time_cycles'].max()

# Convert Series to DataFrame with a specific column name
max_cycle_df = max_value_cycle.to_frame(name='max_cycle')
max_cycle_df.head()

,max_cycle
unit_nr,
1,192
2,287
3,179
4,189
5,269


In [6]:
data_df = data_df.merge(max_cycle_df, left_on='unit_nr', right_index=True)
data_df.head()       

,unit_nr,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,s_5,...,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21,max_cycle
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,192
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,192
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,192
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,192
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,192


In [7]:
data_df['RUL'] = (
    data_df['max_cycle'] - data_df['time_cycles']
)
data_df.head()

,unit_nr,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,s_5,...,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21,max_cycle,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,192,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,192,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,192,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,192,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,192,187


In [8]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20631 entries, 0 to 20630
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   unit_nr      20631 non-null  int64  
 1   time_cycles  20631 non-null  int64  
 2   setting_1    20631 non-null  float64
 3   setting_2    20631 non-null  float64
 4   setting_3    20631 non-null  float64
 5   s_1          20631 non-null  float64
 6   s_2          20631 non-null  float64
 7   s_3          20631 non-null  float64
 8   s_4          20631 non-null  float64
 9   s_5          20631 non-null  float64
 10  s_6          20631 non-null  float64
 11  s_7          20631 non-null  float64
 12  s_8          20631 non-null  float64
 13  s_9          20631 non-null  float64
 14  s_10         20631 non-null  float64
 15  s_11         20631 non-null  float64
 16  s_12         20631 non-null  float64
 17  s_13         20631 non-null  float64
 18  s_14         20631 non-null  float64
 19  s_15

In [9]:
#train_df = data_df[setting_names].copy()
#train_df.head(200)

In [10]:
#train_df.info()

In [11]:
from sklearn.preprocessing import MinMaxScaler


not_scaled_cols = ['unit_nr', 'RUL', 'max_cycle']

col_set = set(col_names)
columns_to_scale = [col for col in col_names if col not in not_scaled_cols]

print("Columns to scale:", columns_to_scale)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(data_df[columns_to_scale])
scaled_data_df = pd.DataFrame(scaled, columns=columns_to_scale, index=data_df.index)

scaled_data_df.insert(0, 'unit_nr', data_df['unit_nr'])
scaled_data_df.insert(len(scaled_data_df.columns), 'RUL', data_df['RUL'])
scaled_data_df.insert(len(scaled_data_df.columns), 'max_cycle', data_df['max_cycle'])
scaled_data_df.head()


Columns to scale: ['time_cycles', 'setting_1', 'setting_2', 'setting_3', 's_1', 's_2', 's_3', 's_4', 's_5', 's_6', 's_7', 's_8', 's_9', 's_10', 's_11', 's_12', 's_13', 's_14', 's_15', 's_16', 's_17', 's_18', 's_19', 's_20', 's_21']


,unit_nr,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,s_5,...,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21,RUL,max_cycle
0,1,0.00000,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,192
1,1,0.00277,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,192
2,1,0.00554,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,192
3,1,0.00831,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,192
4,1,0.01108,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,192


In [12]:
#train_df_scaled = pd.DataFrame(scaled_features, columns=columns_to_scale)
#train_df_scaled.head()

In [13]:
import numpy as np

sequence_length = 50

def create_sequences(data, targets, seq_length=50):
    sequences = []
    target_list = []
    for i in range(len(data) - seq_length + 1):
        seq = data[i:i + seq_length]
        target = targets[i + seq_length - 1]
        sequences.append(seq)
        target_list.append(target)
    return np.array(sequences), np.array(target_list)


In [14]:
engine_ids = scaled_data_df['unit_nr'].unique()
engine_ids

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

In [15]:
train_ids = engine_ids[:80]
test_ids = engine_ids[80:]
print("len train ids:", len(train_ids))
print("len test ids:", len(test_ids))
print("len engine ids:", len(engine_ids))

len train ids: 80
len test ids: 20
len engine ids: 100


In [16]:
#unit_id = engine_ids[0]
#subset = scaled_data_df[scaled_data_df['unit_nr'] == unit_id]
#print(subset.shape)
#subset.head()

In [17]:
train_df = scaled_data_df[scaled_data_df['unit_nr'].isin(train_ids)]
test_df = scaled_data_df[scaled_data_df['unit_nr'].isin(test_ids)]
print("Train df shape:", train_df.shape)
print("Test df shape:", test_df.shape)

train_df.head()

Train df shape: (16138, 28)
Test df shape: (4493, 28)


,unit_nr,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,s_5,...,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21,RUL,max_cycle
0,1,0.00000,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,...,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,192
1,1,0.00277,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,...,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,192
2,1,0.00554,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,...,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,192
3,1,0.00831,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,...,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,192
4,1,0.01108,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,...,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,192


In [18]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

def create_sequences_vectorized(X, y, unit_ids, seq_length=50):
    # 1. Create windows (Batch, Seq, Features)
    X_windows = sliding_window_view(X, window_shape=seq_length, axis=0)
    X_windows = X_windows.transpose(0, 2, 1) # (Batch, Seq, Features)
    
    # 2. Align Targets (End of window)
    y_aligned = y[seq_length-1:]
    
    # 3. Create Mask (Ensure window doesn't cross units)
    unit_ids_start = unit_ids[:-seq_length+1]
    unit_ids_end   = unit_ids[seq_length-1:]
    valid_mask = (unit_ids_start == unit_ids_end)
    
    return X_windows[valid_mask], y_aligned[valid_mask]

# --- Usage ---
sensor_cols = [c for c in train_df.columns if c.startswith('s_')]




# 1. Prepare arrays (Sorted)
train_df = train_df.sort_values(['unit_nr', 'time_cycles'])
test_df = test_df.sort_values(['unit_nr', 'time_cycles'])


# Apply rolling mean with window 9 (common for FD001)
#print("Smoothing sensor data...")
#for col in sensor_cols:
 #   train_df[col] = train_df.groupby('unit_nr')[col].transform(
  #      lambda x: x.rolling(window=9, min_periods=1).mean())
   # test_df[col] = test_df.groupby('unit_nr')[col].transform(
    #    lambda x: x.rolling(window=9, min_periods=1).mean())


# 2. CRITICAL: Drop Target (RUL) and max_cycle from Inputs
# We only want the 24 sensor/setting columns + time_cycle
features_to_drop = ['unit_nr', 'RUL', 'max_cycle', "s_1", "s_5", "s_10", "s_16", "s_18", "s_19"]

X_train_arr = train_df.drop(columns=features_to_drop).values
y_train_arr = train_df['RUL'].values 
train_units = train_df['unit_nr'].values

X_test_arr = test_df.drop(columns=features_to_drop).values
y_test_arr = test_df['RUL'].values
test_units = test_df['unit_nr'].values

# 3. Create Sequences
X_train_seq, y_train_seq = create_sequences_vectorized(X_train_arr, y_train_arr, train_units, 50)
X_test_seq, y_test_seq = create_sequences_vectorized(X_test_arr, y_test_arr, test_units, 50)

print(f"Train Input Shape: {X_train_seq.shape}") 
# Expected shape: (N, 50, 24) -> 24 features

import torch


X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32)   

print(f"X_train_tensor.shape: {X_train_tensor.shape}")
print(f"y_train_tensor.shape: {y_train_tensor.shape}")

print(f"X_test_tensor.shape: {X_test_tensor.shape}")
print(f"y_test_tensor.shape: {y_test_tensor.shape}")
print(f"X_test_tensor[0]: {X_test_tensor[0]}")
print(f"y_test_tensor[0]: {y_test_tensor[0]}")

Train Input Shape: (12218, 50, 19)
X_train_tensor.shape: torch.Size([12218, 50, 19])
y_train_tensor.shape: torch.Size([12218])
X_test_tensor.shape: torch.Size([3513, 50, 19])
y_test_tensor.shape: torch.Size([3513])
X_test_tensor[0]: tensor([[0.0000, 0.2126, 0.7500, 0.0000, 0.2500, 0.4114, 0.4115, 1.0000, 0.6329,
         0.3030, 0.1739, 0.4048, 0.7249, 0.2353, 0.1798, 0.4640, 0.2500, 0.5659,
         0.6107],
        [0.0028, 0.6322, 0.6667, 0.0000, 0.4337, 0.3316, 0.4326, 1.0000, 0.6200,
         0.2879, 0.1911, 0.3214, 0.7207, 0.2794, 0.2086, 0.5094, 0.3333, 0.5969,
         0.6228],
        [0.0055, 0.4713, 0.9167, 0.0000, 0.4036, 0.3353, 0.2389, 1.0000, 0.5523,
         0.3182, 0.1657, 0.3214, 0.5075, 0.2647, 0.2054, 0.4898, 0.5000, 0.6977,
         0.6446],
        [0.0083, 0.4943, 0.5000, 0.0000, 0.3614, 0.5199, 0.3817, 1.0000, 0.7085,
         0.2121, 0.1562, 0.4405, 0.6375, 0.2353, 0.2280, 0.4440, 0.3333, 0.4884,
         0.7269],
        [0.0111, 0.6379, 0.6667, 0.0000, 0.6627

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from sklearn.metrics import mean_squared_error
import copy

# 1. Add Dropout to the Model
class EngineRULPredictor(nn.Module):
    def __init__(self, input_size, hidden_size=256, num_layers=2, dropout=0.2):
        super().__init__()
        # Dropout only works if num_layers > 1
        self.lstm = nn.LSTM(
            input_size, 
            hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            dropout=dropout
        )
        #self.fc = nn.Linear(hidden_size, 1)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        out, _ = self.lstm(x)
        last_out = out[:, -1, :]
        return self.fc(last_out)

# 2. Setup (Reduced hidden size slightly to 128 to prevent overfitting)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(
    TensorDataset(X_train_tensor, y_train_tensor),
    batch_size=64,
    shuffle=True,
    drop_last=True
)

model = EngineRULPredictor(input_size=X_train_tensor.shape[2], hidden_size=128, num_layers=2, dropout=0.3)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)

# 3. Training with "Save Best" logic
EPOCHS = 150  # Lower epochs, let early stopping do the work
best_test_rmse = float('inf')
best_model_wts = copy.deepcopy(model.state_dict())

# Move validation data to GPU once
X_test_gpu = X_test_tensor.to(device)
y_test_real = y_test_seq # Keep real values for RMSE calculation

print("Starting Training with Validation...")

for epoch in range(EPOCHS):
    model.train() # Set to training mode (enables Dropout)
    epoch_loss = 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        out = model(X_batch)
        loss = criterion(out.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    # --- Validation Step ---
    model.eval() # Set to eval mode (disables Dropout)
    with torch.no_grad():
        # Get predictions
        preds_scaled = model(X_test_gpu).cpu().numpy().flatten()
        # Unscale
        preds_real = preds_scaled * max_rul
        # Calculate true RMSE
        mse = mean_squared_error(y_test_real, preds_real)
        current_rmse = np.sqrt(mse)
        scheduler.step(current_rmse)
    
    # Save model if it's the best so far
    if current_rmse < best_test_rmse:
        best_test_rmse = current_rmse
        best_model_wts = copy.deepcopy(model.state_dict())
        print(f"Epoch {epoch+1}: New Best RMSE: {current_rmse:.2f}")
    
    if (epoch + 1) % 10 == 0:
        avg_train_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1}: Train Loss {avg_train_loss:.6f} | Test RMSE {current_rmse:.2f}")

# 4. Load the best weights back
print(f"Training complete. Best RMSE: {best_test_rmse:.2f}")
model.load_state_dict(best_model_wts)

Starting Training with Validation...


NameError: name 'train_loader' is not defined

In [ ]:
X_test_tensor = X_test_tensor.to("cpu")
y_test_tensor = y_test_tensor.to("cpu")
model.to("cpu")

#X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
# y_test_seq is the original TRUE RUL (not scaled)

model.eval()
with torch.no_grad():
    # Predict (Output is 0-1)
    preds_scaled = model(X_test_tensor)
    # Un-scale (Output becomes 0-300)
    preds_real = preds_scaled.cpu().numpy().flatten() * max_rul

# Calculate RMSE on real values
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test_seq, preds_real)
print(f"Test RMSE: {np.sqrt(mse):.2f}")

from matplotlib import pyplot as plt

# Plot
plt.figure(figsize=(10,6))
plt.plot(y_test_seq[:200], label='True RUL')
plt.plot(preds_real[:200], label='Predicted RUL')
plt.legend()
plt.show()

In [ ]:
import torch
print(f'Torch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
print(f'CUDA device count: {torch.cuda.device_count()}')
if torch.cuda.is_available():
    print(f'Active device: {torch.cuda.get_device_name(torch.cuda.current_device())}')
else:
    print('No CUDA devices detected.')

In [ ]:
import torch
if not torch.cuda.is_available():
    raise SystemExit('CUDA is not available; skipping tensor test.')
device = torch.device('cuda')
x = torch.randn(1000, 1000, device=device)
y = torch.randn(1000, 1000, device=device)
z = torch.matmul(x, y)
print(f'Result tensor device: {z.device}')
print(f'Result sample: {z.flatten()[0].item():.6f}')